In [1]:
from promptflow import PFClient

pf_client = PFClient()

from dotenv import load_dotenv

load_dotenv()

runtime = "automatic"
# load flow
flow = "../contoso-chat_hf" #"../contoso-chat"


eval_flow = "multi_flow/"   #"multi_flow_hf/"
data = "../data/salestestdata.jsonl"


In [2]:
# Add a question to test the base prompt flow.
question = "Can you tell me about your jackets?"
customerId = "4"
output_hf = pf_client.test(
    flow=flow, # Path to the flow directory
    inputs={ # Inputs to the flow
        "chat_history": [],
        "question": question,
        "customerId": customerId,
    },
)

output_hf["answer"] = "".join(list(output_hf["answer"]))

[2024-06-05 03:57:07 +0200][promptflow.core._connection][WARNING] - Please use connection.secrets[key] to access secrets.


Prompt flow service has started...
2024-06-05 03:57:07 +0200    1767 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-06-05 03:57:07 +0200    1767 execution.flow     INFO     Start to run 5 nodes with concurrency level 16.
2024-06-05 03:57:07 +0200    1767 execution.flow     INFO     Executing node question_embedding. node run id: e54e3acb-96f2-4af8-8882-125fda94709f_question_embedding_0
2024-06-05 03:57:07 +0200    1767 execution.flow     INFO     Executing node customer_lookup. node run id: e54e3acb-96f2-4af8-8882-125fda94709f_customer_lookup_0
2024-06-05 03:57:08 +0200    1767 execution.flow     INFO     Node question_embedding completes.
2024-06-05 03:57:08 +0200    1767 execution.flow     INFO     Executing node retrieve_documentation. node run id: e54e3acb-96f2-4af8-8882-125fda94709f_retrieve_documentation_0
2024-06-05 03:57:08 +0200    1767 execution.flow     INFO     Node customer_lookup completes.
You can view the trace detail from the following URL:

In [3]:
output_hf

{'answer': "Hi there 👋 SaraLee 💕,\n\nAs we've previously discussed 📝 under our Platinum member perks 🔒, I'd like recommend two fantastic outerwear pieces:\n\n1️⃣ For those breezier days ☀\u200d♀🌞 : Our popular *SummitBreezJacket* (# catalog ID = `03`) would complement perfectly alongside previous purchase ✨CozenightSleepingBag✈. Its breathability & adjustables cuff feature provide optimal performance",
 'context': [{'id': '3',
   'title': 'Summit Breeze Jacket',
   'content': "Discover the joy of hiking with MountainStyle's Summit Breeze Jacket. This lightweight jacket is your perfect companion for outdoor adventures. Sporting a trail-ready, windproof design and a water-resistant fabric, it's ready to withstand any weather. The breathable polyester material and adjustable cuffs keep you comfortable, whether you're ascending a mountain or strolling through a park. And its sleek black color adds style to function. The jacket features a full-zip front closure, adjustable hood, and secure 

Test the groundedness of the prompt flow with the answer from the above question.

# Local Evaluation - Multiple Metrics 

Now use the same prompt flow and test it against the Multi Evaluation flow for groundedness, coherence, fluency, and relevance.

In [4]:
print("question",question)
print("context",output_hf["context"])
print("answer", output_hf["answer"])
test_multi_hf = pf_client.test(
    eval_flow,
    inputs={
        "question": question,
        "context": str(output_hf["context"]),
        "answer": output_hf["answer"],
    },
)

question Can you tell me about your jackets?
context [{'id': '3', 'title': 'Summit Breeze Jacket', 'content': "Discover the joy of hiking with MountainStyle's Summit Breeze Jacket. This lightweight jacket is your perfect companion for outdoor adventures. Sporting a trail-ready, windproof design and a water-resistant fabric, it's ready to withstand any weather. The breathable polyester material and adjustable cuffs keep you comfortable, whether you're ascending a mountain or strolling through a park. And its sleek black color adds style to function. The jacket features a full-zip front closure, adjustable hood, and secure zippered pockets. Experience the comfort of its inner lining and the convenience of its packable design. Crafted for night trekkers too, the jacket has reflective accents for enhanced visibility. Rugged yet chic, the Summit Breeze Jacket is more than a hiking essential, it's the gear that inspires you to reach new heights. Choose adventure, choose the Summit Breeze Jac

You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=contoso-chat_hf&uiTraceId=0xfb61ba964e8487d6c0fd613674fdfb2b
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=contoso-chat_hf&uiTraceId=0xbe100f52b72eec9acb305c48fab7bf57
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=contoso-chat_hf&uiTraceId=0xc6bbc49bc57657c0291908ec3ef775ea
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=contoso-chat_hf&uiTraceId=0xbca8931116f5e5700f0689fefcf7de97
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=contoso-chat_hf&uiTraceId=0x0c6b7fbac8565f43716ec20baa8b1c1c
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=contoso-chat_hf&uiTraceId=0xb6734bc3b9e62bf152b5c60be282a4e8
You 

In [5]:
test_multi_hf

{'gpt_coherence': 5.0,
 'gpt_fluency': 5.0,
 'gpt_groundedness': 1.0,
 'gpt_relevance': 2.0}

In [6]:

# get current time stamp for run name
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y_%m_%d_%H%M%S")
run_name_hf = timestamp+"chat_base_run"
print(run_name_hf)

# create base run in Azure Ai Studio
local_base_run_hf = pf_client.run(
    flow=flow,
    data=data,
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
    },
    runtime=runtime,
    # create a display name as current datetime
    display_name=run_name_hf,
    name=run_name_hf
)
print(local_base_run_hf)

2024_06_05_035710chat_base_run


[2024-06-05 03:57:10 +0200][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run 2024_06_05_035710chat_base_run, log path: /Users/yonghuizhu/.promptflow/.runs/2024_06_05_035710chat_base_run/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=2024_06_05_035710chat_base_run
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0x856a52dcde867d9096b07289a7124bd0
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0xc7e3fb078c34e849eda65ed6926ca130
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0x5a558f2a42695f6608557cd68ea27064
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0xa86ad6dcbf2b4dacd17c8f25ced4d6c9
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0x4bb58ef9a4805630dd39cd8c0f65cdde
name: 2024_06_05_035710chat_base_run
created_on

In [7]:
pf_client.stream(local_base_run_hf)
details = pf_client.get_details(local_base_run_hf)
details.head(10)

2024-06-05 03:57:10 +0200    1767 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-06-05 03:57:11 +0200    1767 execution.bulk     INFO     Current system's available memory is 973.25MB, memory consumption of current process is 158.765625MB, estimated available worker count is 973.25/158.765625 = 6
2024-06-05 03:57:11 +0200    1767 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 5, 'estimated_worker_count_based_on_memory_usage': 6}.
2024-06-05 03:57:11 +0200    1767 execution.bulk     INFO     Process name(SpawnProcess-4)-Process id(1788)-Line number(0) start execution.
2024-06-05 03:57:11 +0200    1767 execution.bulk     INFO     Process name(SpawnProcess-3)-Process id(1787)-Line number(1) start execution.
2024-06-05 03:57:11 +0200    1767 execution.bulk     INFO     Process name(SpawnProcess-5)-Process id(1789)-Line number(2)

,inputs.customerId,inputs.question,inputs.line_number,inputs.chat_history,outputs.answer,outputs.context
0,4,tell me about your hiking jackets,0,[],Hi there 👋 SaraLee!\n\nAs one our valued Plati...,"[{'id': '17', 'title': 'RainGuard Hiking Jacke..."
1,1,Do you have any climbing gear?,1,[],"Hi there 👋 Mr.JohnSmith 🙌,\n\nAs we've previou...","[{'id': '9', 'title': 'SummitClimber Backpack'..."
2,3,Can you tell me about your selection of tents?,2,[],Hi there 👋 Mike 🙌!\n\nAs we've previously disc...,"[{'id': '15', 'title': 'SkyView 2-Person Tent'..."
3,6,Do you have any hiking boots?,3,[],"Hi there 👋 Emilly 🙌,\n\nYes indeed we've got f...","[{'id': '4', 'title': 'TrekReady Hiking Boots'..."
4,2,What gear do you recommend for hiking?,4,[],"Hi there 👋 Janiee 🌳,\n\nAs someone who loves h...","[{'id': '10', 'title': 'TrailBlaze Hiking Pant..."


In [8]:

run_name = timestamp+"chat_eval_run"
print(run_name)

run_name_hf = timestamp+"chat_eval_run"
print(run_name_hf)

local_eval_run_variant = pf_client.run(
    flow=eval_flow,
    data=data,  # path to the data file
    run=local_base_run_hf,  # use run as the variant
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
        "context": "${run.outputs.context}",
        # reference the run's output
        "answer": "${run.outputs.answer}",
    },
    runtime=runtime,
    display_name=run_name,
    name=run_name
)

[2024-06-05 03:57:21 +0200][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run 2024_06_05_035710chat_eval_run, log path: /Users/yonghuizhu/.promptflow/.runs/2024_06_05_035710chat_eval_run/logs.txt


2024_06_05_035710chat_eval_run
2024_06_05_035710chat_eval_run
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=2024_06_05_035710chat_eval_run
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0xf534448f5d41f2ff56aa084ed0bb029d
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0xcb21976ee150d8fd666ac442d9626450
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0xe166b679d7e7236f61c1939bea5eecaf
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0x5cccb60baf7d8cac2500f15da652a990
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=multi_flow&uiTraceId=0xc40a3a1bf724e5bb61

In [9]:

metrics_hf = pf_client.get_metrics(local_eval_run_variant)
import json
print(json.dumps(metrics_hf, indent=4))
pf_client.stream(local_eval_run_variant)

{
    "gpt_coherence": 2.4,
    "gpt_coherence_pass_rate(%)": 20.0,
    "gpt_fluency": 2.4,
    "gpt_fluency_pass_rate(%)": 20.0,
    "gpt_groundedness": 2.6,
    "gpt_groundedness_pass_rate(%)": 40.0,
    "gpt_relevance": 1.4,
    "gpt_relevance_pass_rate(%)": 0.0
}
2024-06-05 03:57:21 +0200    1767 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-06-05 03:57:21 +0200    1767 execution.bulk     INFO     Current system's available memory is 1280.53125MB, memory consumption of current process is 145.21875MB, estimated available worker count is 1280.53125/145.21875 = 8
2024-06-05 03:57:21 +0200    1767 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 5, 'estimated_worker_count_based_on_memory_usage': 8}.
2024-06-05 03:57:22 +0200    1767 execution.bulk     INFO     Process name(SpawnProcess-10)-Process id(1820)-Line number(0) star

In [10]:
pf_client.visualize([local_base_run_hf, local_eval_run_variant])

The HTML file is generated at '/private/var/folders/hf/3mqmr8vd3cs309p9_zg2wcsr0000gn/T/pf-visualize-detail-fvrb2cyz.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
